In [133]:
import pymysql
import numpy as np
import pandas as pd
import csv
import streamlit as st
import time

import matplotlib.pyplot as plt
import seaborn as sns

import datetime
import warnings
warnings.filterwarnings('ignore')

In [121]:
connect = pymysql.connect(host='localhost', user='root', password= '4643', db='danawa', charset='utf8mb4')
cur = connect.cursor()

def make_csv(table, condition, save_file_name):
    
    columns = []
    sql = "SHOW FULL COLUMNS FROM {}".format(table)
    cur.execute(sql)
    rows = cur.fetchall()
    
    for i in range(len(rows)):
        columns.append(rows[i][0])
    
    
    sql = 'SELECT * FROM {} WHERE {}'.format(table, condition)
    cur.execute(sql)
    
    rows = cur.fetchall()
    
    rows = list(rows)
    for row in range(len(rows)):
        rows[row] = list(rows[row])
    
    for i in range(len(rows)):
        rows[i][0] = rows[i][0].strftime("%Y-%m-%d")
    
    f = open(save_file_name, 'w', encoding = 'cp949', newline = '')
    wr = csv.writer(f)
    
    wr.writerow(columns)
    
    for i in range(len(rows)):
        wr.writerow(rows[i])
    
    f.close()
    connect.close()
    
    return f

## Modify above code

In [122]:
def import_data(table, start_date, end_date):
    connect = pymysql.connect(host = 'localhost', user = 'root', password = '4643', db = 'danawa', charset = 'utf8mb4')
    cur = connect.cursor()
    
    col_sql = "SHOW FULL COLUMNS FROM {}".format(table)
    cur.execute(col_sql)
    col_info = cur.fetchall()
    
    columns = [col[0] for col in col_info]
    
    sql = "SELECT * FROM {} WHERE DATE(CRAWL_DATE) >= '{}' AND DATE(CRAWL_DATE) <= '{}'".format(table,start_date, end_date)
    print(sql)
    
    cur.execute(sql)
    result = cur.fetchall()
    
    connect.close()
    
    df = pd.DataFrame(result, columns = columns)
    
    return df

In [272]:
df = import_data(table = 'cpu', start_date = '2020-06-01', end_date = '2020-06-02')
df.loc[df['HOUR'] == 16]

SELECT * FROM cpu WHERE DATE(CRAWL_DATE) >= '2020-06-01' AND DATE(CRAWL_DATE) <= '2020-06-02'


,CRAWL_DATE,HOUR,RANKING,NAME,PRICE,RELEASE_DATE
50,2020-06-01,16,1,인텔 코어i9-10세대 10900 (코멧레이크S) (정품),621000,2020-05-01
51,2020-06-01,16,2,AMD 라이젠3-3세대 3300X (마티스) (정품),187000,2020-05-01
52,2020-06-01,16,3,AMD 라이젠5-3세대 3600 (마티스) (정품),249000,2019-07-01
53,2020-06-01,16,4,AMD 라이젠7-3세대 3700X (마티스) (정품),417000,2019-07-01
54,2020-06-01,16,5,인텔 코어i5-9세대 9400F (커피레이크-R) (정품),217000,2019-01-01
55,2020-06-01,16,6,AMD 라이젠5-3세대 3500X (마티스) (멀티팩),197000,2019-10-01
56,2020-06-01,16,7,인텔 코어i3-9세대 9100F (커피레이크-R) (정품),103000,2019-05-01
57,2020-06-01,16,8,인텔 코어i7-10세대 10700K (코멧레이크S) (정품),548000,2020-05-01
58,2020-06-01,16,9,인텔 코어i5-9세대 9600KF (커피레이크-R) (정품),293000,2019-03-01
59,2020-06-01,16,10,AMD 라이젠3-2세대 3200G (피카소) (정품),129000,2019-07-01


In [124]:
df[df['NAME'].str.contains(r'인텔')]

,CRAWL_DATE,HOUR,RANKING,NAME,PRICE,RELEASE_DATE
0,2020-06-01,15,1,인텔 코어i7-10세대 10700K (코멧레이크S) (정품),548000,2020-05-01
4,2020-06-01,15,5,인텔 코어i5-9세대 9400F (커피레이크-R) (정품),216000,2019-01-01
6,2020-06-01,15,7,인텔 코어i3-9세대 9100F (커피레이크-R) (정품),103000,2019-05-01
7,2020-06-01,15,8,인텔 코어i7-10세대 10700K (코멧레이크S) (정품),548000,2020-05-01
8,2020-06-01,15,9,인텔 코어i5-9세대 9600KF (커피레이크-R) (정품),293000,2019-03-01
...,...,...,...,...,...,...
794,2020-06-02,23,45,인텔 코어i5-8세대 8600 (커피레이크) (정품),415000,2018-04-01
795,2020-06-02,23,46,인텔 코어X-시리즈 i9-10980XE Extreme Edition (캐스케이드레이...,0,2019-12-01
797,2020-06-02,23,48,인텔 셀러론 G4930 (커피레이크-R) (정품),53000,2019-09-01
798,2020-06-02,23,49,인텔 코어i9-9세대 9900 (커피레이크-R) (정품),607000,2019-08-01


## Line plot

### About Price

In [216]:
tmp = pd.pivot_table(df, index = ['CRAWL_DATE', 'HOUR'], columns = 'NAME', values = 'PRICE').reset_index()

tmp_time_list = []

for i in range(len(tmp)):
    time = pd.to_datetime(tmp['HOUR'], format = '%H').astype(str).str.split(" ")[i][1]
    
    tmp_time_list.append(time)
    
tmp['DATE'] = tmp['CRAWL_DATE'].astype(str) + " " +tmp_time_list
tmp.drop(columns = ['CRAWL_DATE', 'HOUR'], inplace = True)
tmp.set_index("DATE", inplace = True)
tmp.columns.name = None

In [203]:
st.line_chart(data)

## About Ranking

In [262]:
tmp_time_list = []

for i in range(len(df)):
    time = pd.to_datetime(df['HOUR'], format = "%H").astype(str).str.split(" ")[i][1]
    tmp_time_list.append(time)
    
df['DATE'] = df['CRAWL_DATE'].astype(str) + " " + tmp_time_list
df.drop(columns = ['CRAWL_DATE', 'HOUR'], inplace = True)
df.set_index("DATE", inplace = True)
df.columns.name = None

In [270]:
a = pd.pivot_table(df, index = df.index, columns = 'NAME', values = 'RANKING')
a

NAME,AMD 라이젠 스레드리퍼 3990X (캐슬 픽) (정품),AMD 라이젠3-1세대 2200G (레이븐 릿지) (정품),AMD 라이젠3-2세대 3200G (피카소) (정품),AMD 라이젠3-3세대 3100 (마티스) (정품),AMD 라이젠3-3세대 3300X (마티스) (정품),AMD 라이젠5-2세대 2600 (피나클 릿지) (정품),AMD 라이젠5-2세대 2600X (피나클 릿지) (정품),AMD 라이젠5-2세대 3400G (피카소) (멀티팩),AMD 라이젠5-2세대 3400G (피카소) (정품),AMD 라이젠5-3세대 3500 (마티스) (멀티팩),...,인텔 코어i7-9세대 9700K (커피레이크-R) (정품),인텔 코어i7-9세대 9700KF (커피레이크-R) (정품),인텔 코어i9-10세대 10900 (코멧레이크S) (정품),인텔 코어i9-10세대 10900K (코멧레이크S) (정품),인텔 코어i9-9세대 9900 (커피레이크-R) (정품),인텔 코어i9-9세대 9900K (커피레이크-R) (정품),인텔 코어i9-9세대 9900KF (커피레이크-R) (정품),인텔 코어i9-9세대 9900KS (커피레이크-R) (정품),인텔 펜티엄 골드 G5400 (커피레이크) (정품),인텔 펜티엄 골드 G6400 (코멧레이크S) (정품)
DATE,,,,,,,,,,,,,,,,,,,,,
2020-06-01 15:00:00,26.0,25.0,10.0,23.0,17.0,40.0,46.0,10.0,44.0,15.0,...,13.0,34.0,29.0,14.0,49.0,16.0,50.0,37.0,39.0,33.0
2020-06-01 16:00:00,26.0,25.0,10.0,23.0,9.5,40.0,46.0,18.0,44.0,15.0,...,13.0,34.0,15.0,14.0,49.0,16.0,50.0,37.0,39.0,33.0
2020-06-01 17:00:00,26.0,25.0,10.0,23.0,17.0,40.0,46.0,18.0,44.0,15.0,...,13.0,34.0,29.0,14.0,49.0,16.0,50.0,37.0,39.0,33.0
2020-06-01 19:00:00,26.0,25.0,10.0,23.0,17.0,40.0,46.0,10.0,44.0,15.0,...,13.0,34.0,29.0,14.0,49.0,16.0,50.0,37.0,39.0,33.0
2020-06-01 20:00:00,26.0,25.0,10.0,23.0,17.0,40.0,46.0,10.0,44.0,15.0,...,13.0,34.0,29.0,14.0,49.0,16.0,50.0,37.0,39.0,33.0
2020-06-01 21:00:00,26.0,25.0,10.0,23.0,17.0,40.0,46.0,18.0,44.0,15.0,...,13.0,34.0,29.0,14.0,49.0,16.0,50.0,37.0,39.0,33.0
2020-06-01 22:00:00,26.0,25.0,10.0,23.0,17.0,40.0,46.0,18.0,44.0,15.0,...,13.0,34.0,29.0,14.0,49.0,16.0,50.0,37.0,39.0,33.0
2020-06-01 23:00:00,26.0,25.0,10.0,23.0,9.5,40.0,46.0,18.0,44.0,15.0,...,13.0,34.0,29.0,14.0,49.0,16.0,50.0,37.0,39.0,33.0
2020-06-02 11:00:00,28.0,25.0,9.0,23.0,16.0,40.0,47.0,18.0,44.0,15.0,...,12.0,35.0,27.0,14.0,49.0,17.0,50.0,37.0,38.0,33.0


In [256]:
tmp = pd.pivot_table(df, index = ['CRAWL_DATE','HOUR'], columns = 'NAME', values = 'RANKING').reset_index()
tmp['DATE'] = tmp['CRAWL_DATE'].astype(str) + " " + tmp_time_list
tmp.drop(columns = ['CRAWL_DATE', 'HOUR'], inplace = True)
tmp.set_index("DATE", inplace = True)
tmp.columns.name = None

for col in tmp.columns:
    tmp[col] = tmp[col].astype(int)
    
tmp

,AMD 라이젠 스레드리퍼 3990X (캐슬 픽) (정품),AMD 라이젠3-1세대 2200G (레이븐 릿지) (정품),AMD 라이젠3-2세대 3200G (피카소) (정품),AMD 라이젠3-3세대 3100 (마티스) (정품),AMD 라이젠3-3세대 3300X (마티스) (정품),AMD 라이젠5-2세대 2600 (피나클 릿지) (정품),AMD 라이젠5-2세대 2600X (피나클 릿지) (정품),AMD 라이젠5-2세대 3400G (피카소) (멀티팩),AMD 라이젠5-2세대 3400G (피카소) (정품),AMD 라이젠5-3세대 3500 (마티스) (멀티팩),...,인텔 코어i7-9세대 9700K (커피레이크-R) (정품),인텔 코어i7-9세대 9700KF (커피레이크-R) (정품),인텔 코어i9-10세대 10900 (코멧레이크S) (정품),인텔 코어i9-10세대 10900K (코멧레이크S) (정품),인텔 코어i9-9세대 9900 (커피레이크-R) (정품),인텔 코어i9-9세대 9900K (커피레이크-R) (정품),인텔 코어i9-9세대 9900KF (커피레이크-R) (정품),인텔 코어i9-9세대 9900KS (커피레이크-R) (정품),인텔 펜티엄 골드 G5400 (커피레이크) (정품),인텔 펜티엄 골드 G6400 (코멧레이크S) (정품)
DATE,,,,,,,,,,,,,,,,,,,,,
2020-06-01 15:00:00,26,25,10,23,17,40,46,10,44,15,...,13,34,29,14,49,16,50,37,39,33
2020-06-01 16:00:00,26,25,10,23,9,40,46,18,44,15,...,13,34,15,14,49,16,50,37,39,33
2020-06-01 17:00:00,26,25,10,23,17,40,46,18,44,15,...,13,34,29,14,49,16,50,37,39,33
2020-06-01 19:00:00,26,25,10,23,17,40,46,10,44,15,...,13,34,29,14,49,16,50,37,39,33
2020-06-01 20:00:00,26,25,10,23,17,40,46,10,44,15,...,13,34,29,14,49,16,50,37,39,33
2020-06-01 21:00:00,26,25,10,23,17,40,46,18,44,15,...,13,34,29,14,49,16,50,37,39,33
2020-06-01 22:00:00,26,25,10,23,17,40,46,18,44,15,...,13,34,29,14,49,16,50,37,39,33
2020-06-01 23:00:00,26,25,10,23,9,40,46,18,44,15,...,13,34,29,14,49,16,50,37,39,33
2020-06-02 11:00:00,28,25,9,23,16,40,47,18,44,15,...,12,35,27,14,49,17,50,37,38,33


In [258]:
df

,CRAWL_DATE,HOUR,RANKING,NAME,PRICE,RELEASE_DATE
0,2020-06-01,15,1,인텔 코어i7-10세대 10700K (코멧레이크S) (정품),548000,2020-05-01
1,2020-06-01,15,2,AMD 라이젠5-2세대 3400G (피카소) (멀티팩),176000,2019-12-01
2,2020-06-01,15,3,AMD 라이젠5-3세대 3600 (마티스) (정품),249000,2019-07-01
3,2020-06-01,15,4,AMD 라이젠7-3세대 3700X (마티스) (정품),418000,2019-07-01
4,2020-06-01,15,5,인텔 코어i5-9세대 9400F (커피레이크-R) (정품),216000,2019-01-01
...,...,...,...,...,...,...
795,2020-06-02,23,46,인텔 코어X-시리즈 i9-10980XE Extreme Edition (캐스케이드레이...,0,2019-12-01
796,2020-06-02,23,47,AMD 라이젠5-2세대 2600X (피나클 릿지) (정품),202000,2018-04-01
797,2020-06-02,23,48,인텔 셀러론 G4930 (커피레이크-R) (정품),53000,2019-09-01
798,2020-06-02,23,49,인텔 코어i9-9세대 9900 (커피레이크-R) (정품),607000,2019-08-01


In [257]:
tmp.iloc[1]

AMD 라이젠 스레드리퍼 3990X (캐슬 픽) (정품)                          26
AMD 라이젠3-1세대 2200G (레이븐 릿지) (정품)                         25
AMD 라이젠3-2세대 3200G (피카소) (정품)                            10
AMD 라이젠3-3세대 3100 (마티스) (정품)                             23
AMD 라이젠3-3세대 3300X (마티스) (정품)                             9
AMD 라이젠5-2세대 2600 (피나클 릿지) (정품)                          40
AMD 라이젠5-2세대 2600X (피나클 릿지) (정품)                         46
AMD 라이젠5-2세대 3400G (피카소) (멀티팩)                           18
AMD 라이젠5-2세대 3400G (피카소) (정품)                            44
AMD 라이젠5-3세대 3500 (마티스) (멀티팩)                            15
AMD 라이젠5-3세대 3500X (마티스) (멀티팩)                            6
AMD 라이젠5-3세대 3600 (마티스) (정품)                              3
AMD 라이젠5-3세대 3600X (마티스) (정품)                            19
AMD 라이젠7-3세대 3700X (마티스) (정품)                             4
AMD 라이젠7-3세대 3800X (마티스) (정품)                            21
AMD 라이젠9-3세대 3900X (마티스) (정품)                            12
AMD 라이젠9-3세대 3950X (마티스) (정품)           

In [243]:
len(tmp.iloc[1])

48

In [224]:
def LinePlotPreprocess(data, process_type):
    if process_type == 'price':
        line_df = pd.pivot_table(data, index = ['CRAWL_DATE', 'HOUR'], columns = 'NAME', values = 'PRICE').reset_index()
    
    elif process_type == 'rank':
        line_df = pd.pivot_table(data, index = ['CRAWL_DATE', 'HOUR'], columns = 'NAME', values = 'RANKING').reset_index()
    
    tmp_time_list = []
    for i in range(len(line_df)):
        time = pd.to_datetime(line_df['HOUR'], format = '%H').astype(str).str.split(" ")[i][1]
        tmp_time_list.append(time)

    line_df['DATE'] = line_df['CRAWL_DATE'].astype(str) + " " + tmp_time_list
    line_df.drop(columns = ['CRAWL_DATE', 'HOUR'], inplace = True)
    line_df.set_index("DATE", inplace = True)
    line_df.columns.name = None

    if process_type == 'rank':
        for col in line_df.columns:
            line_df[col] = line_df[col].astype(int)
    
    return line_df